<a href="https://colab.research.google.com/github/DwarfThief/Raspagem-de-dados-para-iniciantes/blob/master/Tutoriais/WorkshopWebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="info">
  <h1 align="center">Workshop de Web Scraping</h1>
  <p align="center">Workshop dado por Brenno Barboza para a turma de Inteligência Artificial 2019.1 da UFRPE. <p align="center"> <a href="https://docs.google.com/presentation/d/1rgTHkoIPy2TRUaax_RVsQ3466otlt4F-d1H9gfCPc-8/edit?usp=sharing"> Apresentação</a> do workshop. </p>
  </p>
</div>

![](https://i.creativecommons.org/l/by/4.0/88x31.png)

- - -

## O que o nosso cliente quer:

* Nosso cliente quer todas as ***quotes*** (citações) do site [quotes](http://quotes.toscrape.com).
* Juntamente com as **Categorias** e **Autores** das citações.

- - -

![](https://media.giphy.com/media/3o6ozAc3eCahwy4Cpq/giphy.gif)

## Primeiros passos:

* Instalar o Scrapy  e criar nossa primeira Spider.

In [0]:
!pip install scrapy

In [0]:
!scrapy genspider quotes toscrape.com

* O código da Spider vai estar assim:

```
# -*- coding: utf-8 -*-
import scrapy


class WorkshopSpider(scrapy.Spider):
    name = 'Workshop'
    allowed_domains = ['toscrape.com']
    start_urls = ['http://toscrape.com/']

    def parse(self, response):
        pass
```


* Vamos precisar mudar o *`start_urls`* para o link inicial onde a Spider irá acessar.
* Agora vamos verificar se a Spider realmente ta acessando o site que queremos, dentro da função `parse` escrevemos *`self.log("Estou vivo, visitei" + response.url)`*
* Logo em seguida rodamos a nossa Spider novamente.

In [0]:
!scrapy runspider quotes.py

- - -
## Agora que sabemos que a nossa Spider ta acessando o site, precisamos começar a coletar o que o cliente deseja.

### Vamos analisar a página.
* *`response.css('<LocalDeMineração>::<OQueQueremos>').<saida>()*`



In [0]:
!scrapy shell http://quotes.toscrape.com/

* Como vamos coletar:
  * Todos os textos das Citações:
    *`response.css('span.text::text').extract()`*
  * Todas as tags por Citação:
    *`response.css('small.author::text').extract()`*
  * Todos os autores das Citações:
    *`response.css('a.tag::text').extract()`*

### Vamos adicionar essa extração na nossa Spider
* Vamos colocar um *yield* e dentro dele inserir as informações que queremos guardar.
* Vamos dar nomes aos valores que vamos guardar.

### Por final vamos rodar a nossa Spider, mas com uma coisa especial.

In [0]:
!scrapy runspider quotes.py -o text.json

- - -

## Melhorando a nossa Spider

### Já que não usar o *`extract_first()`* ao coletar as Tags acabamos pegando as Tags de todas as citações.

* Precisamos saber de onde vêem as Tags.
* Cada categoria ta dentro de um *`<div class="quote">`*
* Vamo ver oq existe dentro dessas quotes.
* Vamos usar o Scrapy Shell novamente e olhar o que existe dentro dessas citações.
  * Escreva *`response.css('div.quote')`*

In [0]:
!scrapy shell http://quotes.toscrape.com/


* Criar um loop para varrer as **citações**
```
  for quote in response.css('div.quote'):
    coletado = {
        'autor': quote.css('small.author::text').extract_first(),
        'texto': quote.css('span.text::text').extract_first(),
        'categorias': quote.css('a.tag::text').extract(),
    }
    yield coletado
```

### Pronto, agora a gente consegue coletar tudo na página.

- - - 

## Agora vamos navegar entre as páginas e coletar tudo do site.

![](https://media.tenor.com/images/f1921d017450cd690cfa73bfe33d7724/tenor.gif)

* Hora de inspecionar o botão de *Next* da página.
* O botão *Next* esta na Tag *`li`* e sua classe é *`next`*, sendo assim vamos extrair dessa maneira:
```response.css('li.next')```
* O link para a próxima página está na subcategoria `<a>`, então vamos ver o que sai se extrairmos isso e a sua String junto. Escreva isso no Scrapy Shell: *`response.css('li.next > a').extract_first()`*
>> O > após o *`li.next`* significa quer queremos o que está dentro dessa subcategoria.

In [0]:
!scrapy shell http://quotes.toscrape.com/

* Dentro da subcategoria `<a>` temos acesso ao *`href`*, que é o responsável pelo funcionamento do botão, é isso que queremos.

>> ```response.css('li.next > a::attr(href)').extract_first()```

> Vamos colocar isso dentro de uma variável e usar essa variável para receber os *`response`* da próxima página, usando o método *`.urljoin()`* com a variável como parâmetro.

>> ```response.urljoin(proxima_pag)```

> Essa função vai retornar um novo url, vamos guardar isso em outra variável.
>> ```proxima_pag = response.css('li.next > a::attr(href)').extract_first()```

>> ```proxima_pag_response = response.urljoin(proxima_pag)```

> Agora vamos mandar o request para o servidor, receber o response e repetir tudo. Adicionando o método *`Request()`*, passando como parâmetro a variável *`proxima_pag_response`* como a url de acesso e a função para a recursividade, no nosso caso, o parse.
>> ```yield scrapy.Request( url = proxima_pag, callback = self.parse)```

- - -

## Nosso código final deve ficar assim:

```
# -*- coding: utf-8 -*-
import scrapy

class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    allowed_domains = ['toscrape.com']
    start_urls = ['http://quotes.toscrape.com']

    def parse(self, response):
        #Extraindo as citações
        for quote in response.css('div.quote'):     
            coletado = {
                'autor': quote.css('small.author::text').extract_first(),
                'texto': quote.css('span.text::text').extract_first(),
                'categorias': quote.css('a.tag::text').extract(),
            }
            yield coletado
        #Navegação entre páginas
        proxima_pag = response.css('li.next > a::attr(href)').extract_first()
        proxima_pag_response = response.urljoin(proxima_pag)
        yield scrapy.Request( url = proxima_pag_response, callback = self.parse)
```

In [0]:
!scrapy runspider quotes -o text.json